# Mushroom Edibility

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing
from sklearn.feature_selection import RFE

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, log_loss
%matplotlib inline

In [48]:
mushroom = pd.read_csv("data/agaricus-lepiota.data.csv")

In [56]:
mushroom = mushroom.apply(preprocessing.LabelEncoder().fit_transform)

In [57]:
# define the target
target = mushroom['p']

# define the classifier data
data = mushroom.iloc[:, 1:23]

In [59]:
# split the training and test data
Xtrain, Xtest, ytrain, ytest = train_test_split(data, target, test_size=0.50, random_state=1)

In [62]:
# First I am trying Naive Bayes
model = GaussianNB()
model.fit(Xtrain, ytrain)

train_err = 1-model.score(Xtrain, ytrain)
print("Training error for this model is %s" % train_err)
test_err = 1-model.score(Xtest, ytest)
print("Test error for this model is %s" % test_err)

Training error for this model is 0.0837232208816
Test error for this model is 0.0839487936977


Not bad...

In [64]:
# Now I'm trying a linear SVM
model = SVC(kernel = 'linear')
model.fit(Xtrain, ytrain)

train_err = 1-model.score(Xtrain, ytrain)
print("Training error for this model is %s" % train_err)
test_err = 1-model.score(Xtest, ytest)
print("Test error for this model is %s" % test_err)

Training error for this model is 0.0376754493967
Test error for this model is 0.0425898572132


This looks like it has an overall better accuracy than the Naive Bayes. 

In [69]:
# Next, I want to tune the model a little bit 
gammas = 10.0**np.arange(-6,1)
parameters = {'gamma':gammas}
svc = SVC(kernel = 'linear')

clf = GridSearchCV(svc, {'gamma':gammas, 'C':[0.01,0.1,1,1.0,10.0]})
clf.fit(Xtrain,ytrain)
print(clf.best_params_)
print(clf.best_score_)

{'C': 10.0, 'gamma': 9.9999999999999995e-07}
0.982516621522


Alright, now that I have these parameters chosen, I'm going to think about the relevant features.

In [90]:
def fit_and_report(model, X, y, Xv, yv):
    model.fit(X,y)
    mean_squared_err = lambda y, yhat: np.mean((y-yhat)**2)
    errors = [mean_squared_err(y, model.predict(X)), mean_squared_err(yv, model.predict(Xv))]
    return errors

In [92]:
# I'm going to use sklearn's `RFE` for this
svc = SVC(kernel = 'linear',gamma = 1e-06, C = 10)

rfe = RFE(estimator = svc, n_features_to_select = 10)

rfe.fit(Xtrain, ytrain)
Xtrain_sel = Xtrain.iloc[:, rfe.support_]
Xtest_sel = Xtest.iloc[:, rfe.support_]

errs = fit_and_report(svc, Xtrain_sel, ytrain, Xtest_sel, ytest)

print("Training error:   %.3g" % errs[0])
print("Validation error: %.3g" % errs[1])

[False False False  True False  True  True  True False  True  True  True
 False False False False  True  True  True False False False]
Training error:   0.0266
Validation error: 0.0286


Ok, but how do i know that I only want 10 features? I'm going to run through the full gamut to decide how many that I want.

In [ ]:
# make an empty list to hold train error, and one to hold test error
trainer = list()
valerr = list()

ft = list(range(1,23))
# loop over the features, and fit the model each time
# also calculate train and test error
for f in ft:
    svc = SVC(kernel = 'linear',gamma = 1e-06, C = 10)

    rfe = RFE(estimator = svc, n_features_to_select = f)

    rfe.fit(Xtrain, ytrain)
    Xtrain_sel = Xtrain.iloc[:, rfe.support_]
    Xtest_sel = Xtest.iloc[:, rfe.support_]
    
    errs = fit_and_report(svc, Xtrain_sel, ytrain, Xtest_sel, ytest)
    
    trainer.append(errs[0])
    valerr.append(errs[1])

    # plot validation loss
plt.plot(ft, trainer, label="train")
plt.plot(ft, valerr, label="test")
plt.xlabel("feats")
plt.ylabel("error")
plt.legend()
plt.show()